<a href="https://colab.research.google.com/github/zanderst/Flask/blob/main/Shallow_learning_models_finalised%3F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This Google Colab sheet provides python code for shallow predictive ML models.  includes models trained on both 17 and 18 feature sets.
This is because only two validation sets had 18 features, validation set 3 was missing HbA1c column.

Several strategies to optimise performance have been explored, and only the optimal remain. Given the unbalanced nature of the datasets, optimisation is challenging.


Smotenc used to oversample minority. logistic regression is polynomial logistic regression which seems to improve overall performance. all use exhaustive hyperparameter search, k fold cross validation.

Check GPU status (to check colab gpu allocation)

In [ ]:
!nvidia-smi

Mon Aug 12 17:01:14 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   46C    P8              11W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

code to mount google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


17 feature model - tested against all three validation sets. trained 10 times with different seed each time - **Used in final report / statsistical analysis**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, average_precision_score, recall_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from imblearn.over_sampling import SMOTENC

# Function to transform categorical columns
def transform_categorical_columns(df):
    df['Sex'] = df['Sex'].replace({2: -1, 1: -1})
    df['DM.IFG'] = df['DM.IFG'].replace({0: -1, 1: -1})
    return df

# function to evaluate the model on a dataset and return metrics
def evaluate_model(model, X, y_true, threshold=0.5):
    y_pred_proba = model.predict_proba(X)[:, 1]
    y_pred = (y_pred_proba >= threshold).astype(int)
    accuracy = accuracy_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)
    cr = classification_report(y_true, y_pred, output_dict=True)
    roc_auc = roc_auc_score(y_true, y_pred_proba)
    aupr = average_precision_score(y_true, y_pred_proba)
    sensitivity = recall_score(y_true, y_pred, pos_label=1)

    f1_0 = cr['0']['f1-score']
    f1_1 = cr['1']['f1-score']

    return {
        'accuracy': accuracy,
        'confusion_matrix': cm,
        'roc_auc': roc_auc,
        'aupr': aupr,
        'sensitivity': sensitivity,
        'f1_0': f1_0,
        'f1_1': f1_1
    }

# Function to perform hyperparameter search with cross-validation
def hyperparameter_search(model, param_grid, X_resampled, y_resampled, random_state):
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=cv, scoring='roc_auc', n_jobs=-1, verbose=2)
    grid_search.fit(X_resampled, y_resampled)
    return grid_search.best_estimator_

# Load the discovery set
discovery_set_path = '/content/drive/MyDrive/Liver Fibrosis/fibrosis_discovery_set.csv'
data = pd.read_csv(discovery_set_path)

# Drop HbA1c
if 'HbA1c' in data.columns:
    data = data.drop(columns=['HbA1c'])

# Apply transformation to the discovery set
data = transform_categorical_columns(data)

# Lists of numerical features and categorical features
numerical_features = ['Age', 'ALB', 'ALT', 'AST', 'BMI', 'FBG', 'GGT', 'HDL', 'LDL', 'PLT', 'TBIL', 'TC', 'TG', 'AST.ALT', 'AST.PLT']
categorical_features = ['Sex', 'DM.IFG']

# Separate features 'x' and target 'y'
X = data.drop(columns=['group'])
y = data['group']

# Standardise numerical features
scaler = StandardScaler()
X[numerical_features] = scaler.fit_transform(X[numerical_features])

# Generate polynomial features for Logistic Regression
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X[numerical_features])
X_poly_df = pd.DataFrame(X_poly, columns=poly.get_feature_names_out(numerical_features))
X_poly_combined = pd.concat([X_poly_df, data[categorical_features].reset_index(drop=True)], axis=1)
X_poly_combined.columns = X_poly_combined.columns.astype(str)

# Combine numerical and categorical features for other models
X_combined = pd.concat([pd.DataFrame(X, columns=numerical_features), data[categorical_features].reset_index(drop=True)], axis=1)
X_combined.columns = X_combined.columns.astype(str)

# Define parameter grids for each model
param_grid_logistic = {
    'C': [0.01, 0.1, 1, 10, 100],
    'class_weight': [{0: 1, 1: 2}, {0: 1, 1: 3}, 'balanced']
}

param_grid_random_forest = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

param_grid_gradient_boosting = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5]
}

param_grid_decision_tree = {
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialise models
models = {
    'Logistic Regression': LogisticRegression(max_iter=3000),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'Decision Tree': DecisionTreeClassifier()
}

param_grids = {
    'Logistic Regression': param_grid_logistic,
    'Random Forest': param_grid_random_forest,
    'Gradient Boosting': param_grid_gradient_boosting,
    'Decision Tree': param_grid_decision_tree
}

all_results = {
    'discovery_set': {},
    'validation_set1': {},
    'validation_set2': {},
    'validation_set3': {}
}

for model_name in models:
    for dataset in all_results:
        all_results[dataset][model_name] = {
            'accuracy': [],
            'roc_auc': [],
            'aupr': [],
            'sensitivity': [],
            'f1_0': [],
            'f1_1': []
        }

# complete hyperparameter search, train models 10 times, and evaluate
for run in range(1, 11):
    print(f"Run {run}")

    # Apply SMOTENC to balance the dataset with different random state each run
    smotenc = SMOTENC(categorical_features=[X_poly_combined.columns.get_loc(col) for col in categorical_features], random_state=run)
    X_resampled_poly, y_resampled_poly = smotenc.fit_resample(X_poly_combined, y)

    smotenc = SMOTENC(categorical_features=[X_combined.columns.get_loc(col) for col in categorical_features], random_state=run)
    X_resampled_combined, y_resampled_combined = smotenc.fit_resample(X_combined, y)

    # Initialise models with different random state each run
    models = {
        'Logistic Regression': LogisticRegression(max_iter=3000, random_state=run),
        'Random Forest': RandomForestClassifier(random_state=run),
        'Gradient Boosting': GradientBoostingClassifier(random_state=run),
        'Decision Tree': DecisionTreeClassifier(random_state=run)
    }

    best_models = {}

    # Perform hyperparameter search and fit models
    for model_name in models:
        print(f"Training and hyperparameter tuning for {model_name}...")
        if model_name == 'Logistic Regression':
            best_model = hyperparameter_search(models[model_name], param_grids[model_name], X_resampled_poly, y_resampled_poly, random_state=run)
        else:
            best_model = hyperparameter_search(models[model_name], param_grids[model_name], X_resampled_combined, y_resampled_combined, random_state=run)

        best_models[model_name] = best_model

        # Evaluate models with specified thresholds
        if model_name == 'Logistic Regression':
            threshold = 0.5
            X_eval = X_poly_combined
        else:
            threshold = 0.5
            X_eval = X_combined

        print(f"Evaluation on the discovery set for {model_name} with threshold {threshold}:")
        discovery_metrics = evaluate_model(best_model, X_eval, y, threshold=threshold)
        for metric in all_results['discovery_set'][model_name]:
            all_results['discovery_set'][model_name][metric].append(discovery_metrics[metric])

        # Print metrics for each run
        print(f"Run {run} - Discovery Set - {model_name} Metrics:")
        for metric_name, metric_value in discovery_metrics.items():
            print(f"{metric_name}: {metric_value}")

    # Paths to the validation sets
    validation_sets = {
        'validation_set1': '/content/drive/MyDrive/Liver Fibrosis/validation_set1.csv',
        'validation_set2': '/content/drive/MyDrive/Liver Fibrosis/validation_set2.csv',
        'validation_set3': '/content/drive/MyDrive/Liver Fibrosis/validation_set3.csv'
    }

    # Evaluate each best model on each validation set
    for validation_set_name, validation_set_path in validation_sets.items():
        validation_data = pd.read_csv(validation_set_path)

        # Drop HbA1c for validation sets
        if 'HbA1c' in validation_data.columns:
            validation_data = validation_data.drop(columns=['HbA1c'])

        # Apply the transformation to the validation set
        validation_data = transform_categorical_columns(validation_data)

        # Separate features (X_val) and target (y_val)
        X_val = validation_data.drop(columns=['group'])
        y_val = validation_data['group']

        # Standardise numerical features using the same scaler fitted on the discovery set
        X_val[numerical_features] = scaler.transform(X_val[numerical_features])

        # Generate polynomial features for Logistic Regression
        X_val_poly = poly.transform(X_val[numerical_features])
        X_val_poly_df = pd.DataFrame(X_val_poly, columns=poly.get_feature_names_out(numerical_features))
        X_val_poly_combined = pd.concat([X_val_poly_df, validation_data[categorical_features].reset_index(drop=True)], axis=1)
        X_val_poly_combined.columns = X_val_poly_combined.columns.astype(str)

        # Combine numerical and categorical features for other models
        X_val_combined = pd.concat([pd.DataFrame(X_val, columns=numerical_features), validation_data[categorical_features].reset_index(drop=True)], axis=1)
        X_val_combined.columns = X_val_combined.columns.astype(str)

        for model_name in best_models:
            if model_name == 'Logistic Regression':
                threshold = 0.50
                X_eval = X_val_poly_combined
            else:
                threshold = 0.5
                X_eval = X_val_combined

            print(f"Evaluation on {validation_set_name} for {model_name} with threshold {threshold}:")
            validation_metrics = evaluate_model(best_models[model_name], X_eval, y_val, threshold=threshold)
            for metric in all_results[validation_set_name][model_name]:
                all_results[validation_set_name][model_name][metric].append(validation_metrics[metric])

            # Print metrics for each run
            print(f"Run {run} - {validation_set_name} - {model_name} Metrics:")
            for metric_name, metric_value in validation_metrics.items():
                print(f"{metric_name}: {metric_value}")

# Calculate mean and standard dev for each metric
final_results = {}

for dataset_name, dataset_results in all_results.items():
    final_results[dataset_name] = {}
    for model_name, metrics in dataset_results.items():
        final_results[dataset_name][model_name] = {}
        for metric_name, values in metrics.items():
            final_results[dataset_name][model_name][f'{metric_name}_mean'] = np.mean(values)
            final_results[dataset_name][model_name][f'{metric_name}_std'] = np.std(values)

# Print final results
for dataset_name, dataset_results in final_results.items():
    print(f"Final Results for {dataset_name}:")
    for model_name, metrics in dataset_results.items():
        print(f"Model: {model_name}")
        for metric_name, value in metrics.items():
            print(f"{metric_name}: {value}")
        print("\n")


Run 1
Training and hyperparameter tuning for Logistic Regression...
Fitting 5 folds for each of 15 candidates, totalling 75 fits
Evaluation on the discovery set for Logistic Regression with threshold 0.5:
Run 1 - Discovery Set - Logistic Regression Metrics:
accuracy: 0.8666666666666667
confusion_matrix: [[341  50]
 [ 22 127]]
roc_auc: 0.9456221356356959
aupr: 0.8863469670913571
sensitivity: 0.8523489932885906
f1_0: 0.9045092838196287
f1_1: 0.7791411042944785
Training and hyperparameter tuning for Random Forest...
Fitting 5 folds for each of 81 candidates, totalling 405 fits
Evaluation on the discovery set for Random Forest with threshold 0.5:
Run 1 - Discovery Set - Random Forest Metrics:
accuracy: 1.0
confusion_matrix: [[391   0]
 [  0 149]]
roc_auc: 1.0
aupr: 0.9999999999999999
sensitivity: 1.0
f1_0: 1.0
f1_1: 1.0
Training and hyperparameter tuning for Gradient Boosting...
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Evaluation on the discovery set for Gradient Boost

18 feature model - prints out each run and metrics (only compatible with validation set 1 and 2)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, average_precision_score, recall_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from imblearn.over_sampling import SMOTENC

#function to transform categorical columns
def transform_categorical_columns(df):
    df['Sex'] = df['Sex'].replace({2: -1, 1: 1})
    df['DM.IFG'] = df['DM.IFG'].replace({0: -1, 1: 1})
    return df

# Function to evaluate the model on a dataset and return metrics
def evaluate_model(model, X, y_true, threshold=0.5):
    y_pred_proba = model.predict_proba(X)[:, 1]
    y_pred = (y_pred_proba >= threshold).astype(int)
    accuracy = accuracy_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)
    cr = classification_report(y_true, y_pred, output_dict=True)
    roc_auc = roc_auc_score(y_true, y_pred_proba)
    aupr = average_precision_score(y_true, y_pred_proba)
    sensitivity = recall_score(y_true, y_pred, pos_label=1)

    f1_0 = cr['0']['f1-score']
    f1_1 = cr['1']['f1-score']

    return {
        'accuracy': accuracy,
        'confusion_matrix': cm,
        'roc_auc': roc_auc,
        'aupr': aupr,
        'sensitivity': sensitivity,
        'f1_0': f1_0,
        'f1_1': f1_1
    }

# Function to perform hyperparameter search with cross-validation
def hyperparameter_search(model, param_grid, X_resampled, y_resampled, random_state):
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=cv, scoring='roc_auc', n_jobs=-1, verbose=2)
    grid_search.fit(X_resampled, y_resampled)
    return grid_search.best_estimator_

# Load discovery set
discovery_set_path = '/content/drive/MyDrive/Liver Fibrosis/fibrosis_discovery_set.csv'
data = pd.read_csv(discovery_set_path)

# Apply the transformation to discovery set
data = transform_categorical_columns(data)

# Lists of numerical features and categorical features
numerical_features = ['Age', 'ALB', 'ALT', 'AST', 'BMI', 'FBG', 'GGT', 'HbA1c', 'HDL', 'LDL', 'PLT', 'TBIL', 'TC', 'TG', 'AST.ALT', 'AST.PLT']
categorical_features = ['Sex', 'DM.IFG']

# Separate features 'x' and 'y'
X = data.drop(columns=['group'])
y = data['group']

# Standardise numerical features
scaler = StandardScaler()
X[numerical_features] = scaler.fit_transform(X[numerical_features])

# Make polynomial features for Logistic Regression
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X[numerical_features])
X_poly_df = pd.DataFrame(X_poly, columns=poly.get_feature_names_out(numerical_features))
X_poly_combined = pd.concat([X_poly_df, data[categorical_features].reset_index(drop=True)], axis=1)
X_poly_combined.columns = X_poly_combined.columns.astype(str)

# Combine numerical and categorical features for other models
X_combined = pd.concat([pd.DataFrame(X, columns=numerical_features), data[categorical_features].reset_index(drop=True)], axis=1)
X_combined.columns = X_combined.columns.astype(str)

# Define parameter grids for each model
param_grid_logistic = {
    'C': [0.01, 0.1, 1, 10, 100],
    'class_weight': [{0: 1, 1: 2}, {0: 1, 1: 3}, 'balanced']
}

param_grid_random_forest = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

param_grid_gradient_boosting = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5]
}

param_grid_decision_tree = {
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialise models
models = {
    'Logistic Regression': LogisticRegression(max_iter=3000),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'Decision Tree': DecisionTreeClassifier()
}

param_grids = {
    'Logistic Regression': param_grid_logistic,
    'Random Forest': param_grid_random_forest,
    'Gradient Boosting': param_grid_gradient_boosting,
    'Decision Tree': param_grid_decision_tree
}

all_results = {
    'discovery_set': {},
    'validation_set1': {},
    'validation_set2': {}
}

for model_name in models:
    for dataset in all_results:
        all_results[dataset][model_name] = {
            'accuracy': [],
            'roc_auc': [],
            'aupr': [],
            'sensitivity': [],
            'f1_0': [],
            'f1_1': []
        }

# Complete hyperparameter search, train models 10 times, and evaluate
for run in range(1, 11):
    print(f"Run {run}")

    # Apply SMOTENC to balance the dataset with different random state each run
    smotenc = SMOTENC(categorical_features=[X_poly_combined.columns.get_loc(col) for col in categorical_features], random_state=run)
    X_resampled_poly, y_resampled_poly = smotenc.fit_resample(X_poly_combined, y)

    smotenc = SMOTENC(categorical_features=[X_combined.columns.get_loc(col) for col in categorical_features], random_state=run)
    X_resampled_combined, y_resampled_combined = smotenc.fit_resample(X_combined, y)

    # Initialise models with different random state each run
    models = {
        'Logistic Regression': LogisticRegression(max_iter=3000, random_state=run),
        'Random Forest': RandomForestClassifier(random_state=run),
        'Gradient Boosting': GradientBoostingClassifier(random_state=run),
        'Decision Tree': DecisionTreeClassifier(random_state=run)
    }

    best_models = {}

    # hyperparameter search and fit models
    for model_name in models:
        print(f"Training and hyperparameter tuning for {model_name}...")
        if model_name == 'Logistic Regression':
            best_model = hyperparameter_search(models[model_name], param_grids[model_name], X_resampled_poly, y_resampled_poly, random_state=run)
        else:
            best_model = hyperparameter_search(models[model_name], param_grids[model_name], X_resampled_combined, y_resampled_combined, random_state=run)

        best_models[model_name] = best_model

        # Evaluate models with specified thresholds
        if model_name == 'Logistic Regression':
            threshold = 0.55
            X_eval = X_poly_combined
        else:
            threshold = 0.5
            X_eval = X_combined

        print(f"Evaluation on the discovery set for {model_name} with threshold {threshold}:")
        discovery_metrics = evaluate_model(best_model, X_eval, y, threshold=threshold)
        for metric in all_results['discovery_set'][model_name]:
            all_results['discovery_set'][model_name][metric].append(discovery_metrics[metric])

        # Print metrics for each run
        print(f"Run {run} - Discovery Set - {model_name} Metrics:")
        for metric_name, metric_value in discovery_metrics.items():
            print(f"{metric_name}: {metric_value}")

    # Paths to the validation sets
    validation_sets = {
        'validation_set1': '/content/drive/MyDrive/Liver Fibrosis/validation_set1.csv',
        'validation_set2': '/content/drive/MyDrive/Liver Fibrosis/validation_set2.csv'
    }

    # Evaluate each best model on each validation set
    for validation_set_name, validation_set_path in validation_sets.items():
        validation_data = pd.read_csv(validation_set_path)

        # Apply the transformation to the validation set
        validation_data = transform_categorical_columns(validation_data)

        # Separate features 'X_val' and target 'y_val'
        X_val = validation_data.drop(columns=['group'])
        y_val = validation_data['group']

        # Standardise numerical features using the same scaler fitted on the discovery set
        X_val[numerical_features] = scaler.transform(X_val[numerical_features])

        # Generate polynomial features for Logistic Regression
        X_val_poly = poly.transform(X_val[numerical_features])
        X_val_poly_df = pd.DataFrame(X_val_poly, columns=poly.get_feature_names_out(numerical_features))
        X_val_poly_combined = pd.concat([X_val_poly_df, validation_data[categorical_features].reset_index(drop=True)], axis=1)
        X_val_poly_combined.columns = X_val_poly_combined.columns.astype(str)

        # Combine numerical and categorical features for other models
        X_val_combined = pd.concat([pd.DataFrame(X_val, columns=numerical_features), validation_data[categorical_features].reset_index(drop=True)], axis=1)
        X_val_combined.columns = X_val_combined.columns.astype(str)

        for model_name in best_models:
            if model_name == 'Logistic Regression':
                threshold = 0.55
                X_eval = X_val_poly_combined
            else:
                threshold = 0.5
                X_eval = X_val_combined

            print(f"Evaluation on {validation_set_name} for {model_name} with threshold {threshold}:")
            validation_metrics = evaluate_model(best_models[model_name], X_eval, y_val, threshold=threshold)
            for metric in all_results[validation_set_name][model_name]:
                all_results[validation_set_name][model_name][metric].append(validation_metrics[metric])

            # Print metrics for each run
            print(f"Run {run} - {validation_set_name} - {model_name} Metrics:")
            for metric_name, metric_value in validation_metrics.items():
                print(f"{metric_name}: {metric_value}")

# Calculate mean and standard deviation for each metric
final_results = {}

for dataset_name, dataset_results in all_results.items():
    final_results[dataset_name] = {}
    for model_name, metrics in dataset_results.items():
        final_results[dataset_name][model_name] = {}
        for metric_name, values in metrics.items():
            final_results[dataset_name][model_name][f'{metric_name}_mean'] = np.mean(values)
            final_results[dataset_name][model_name][f'{metric_name}_std'] = np.std(values)

# Print final results
for dataset_name, dataset_results in final_results.items():
    print(f"Final Results for {dataset_name}:")
    for model_name, metrics in dataset_results.items():
        print(f"Model: {model_name}")
        for metric_name, value in metrics.items():
            print(f"{metric_name}: {value}")
        print("\n")


Run 1
Training and hyperparameter tuning for Logistic Regression...
Fitting 5 folds for each of 15 candidates, totalling 75 fits
Evaluation on the discovery set for Logistic Regression with threshold 0.55:
Run 1 - Discovery Set - Logistic Regression Metrics:
accuracy: 0.8388888888888889
confusion_matrix: [[338  53]
 [ 34 115]]
roc_auc: 0.9226213975523097
aupr: 0.849521205913931
sensitivity: 0.7718120805369127
f1_0: 0.8859764089121887
f1_1: 0.7255520504731862
Training and hyperparameter tuning for Random Forest...
Fitting 5 folds for each of 81 candidates, totalling 405 fits
Evaluation on the discovery set for Random Forest with threshold 0.5:
Run 1 - Discovery Set - Random Forest Metrics:
accuracy: 1.0
confusion_matrix: [[391   0]
 [  0 149]]
roc_auc: 1.0
aupr: 1.0
sensitivity: 1.0
f1_0: 1.0
f1_1: 1.0
Training and hyperparameter tuning for Gradient Boosting...
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Evaluation on the discovery set for Gradient Boosting with thresh

Using the limited feature selection (outline by Sang et al in Diagnosis of fibrosis using blood markers and logistic regression in southeast asian patients with non-alcoholic fatty liver disease) - interestingly, the features they outlined indeed did improve performance on validation set 1 and 2, but performed significantly worse on validation set 3. For this reason, we will be optimising based on all 17 available biomarkers.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, average_precision_score, recall_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from imblearn.over_sampling import SMOTENC

# Function to transform categorical columns
def transform_categorical_columns(df):
    df['Sex'] = df['Sex'].replace({2: -1, 1: 1})
    df['DM.IFG'] = df['DM.IFG'].replace({0: -1, 1: 1})
    return df

# function to evaluate the model on a dataset and return metrics including sensitivity
def evaluate_model(model, X, y_true, threshold=0.5):
    y_pred_proba = model.predict_proba(X)[:, 1]
    y_pred = (y_pred_proba >= threshold).astype(int)
    accuracy = accuracy_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)
    cr = classification_report(y_true, y_pred, output_dict=True)
    roc_auc = roc_auc_score(y_true, y_pred_proba)
    aupr = average_precision_score(y_true, y_pred_proba)
    sensitivity = recall_score(y_true, y_pred, pos_label=1)

    f1_0 = cr['0']['f1-score']
    f1_1 = cr['1']['f1-score']

    return {
        'accuracy': accuracy,
        'confusion_matrix': cm,
        'roc_auc': roc_auc,
        'aupr': aupr,
        'sensitivity': sensitivity,
        'f1_0': f1_0,
        'f1_1': f1_1
    }

# Function to complete hyperparameter search with cross-validation
def hyperparameter_search(model, param_grid, X_resampled, y_resampled, random_state):
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=cv, scoring='roc_auc', n_jobs=-1, verbose=2)
    grid_search.fit(X_resampled, y_resampled)
    return grid_search.best_estimator_

# Load the discovery set
discovery_set_path = '/content/drive/MyDrive/Liver Fibrosis/fibrosis_discovery_set.csv'
data = pd.read_csv(discovery_set_path)

# Apply transformation to the discovery set
data = transform_categorical_columns(data)

# Lists of selected numerical features and categorical features
numerical_features = ['Age', 'ALT', 'BMI', 'FBG', 'GGT', 'TG', 'AST.PLT']
categorical_features = ['DM.IFG']

# Drop all other columns except the 7 selected features
features_to_keep = numerical_features + categorical_features + ['group']
data = data[features_to_keep]

# Separate features 'x' and target 'y'
X = data.drop(columns=['group'])
y = data['group']

# Standardise numerical features
scaler = StandardScaler()
X[numerical_features] = scaler.fit_transform(X[numerical_features])

# Generate polynomial features for Logistic Regression
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X[numerical_features])
X_poly_df = pd.DataFrame(X_poly, columns=poly.get_feature_names_out(numerical_features))
X_poly_combined = pd.concat([X_poly_df, data[categorical_features].reset_index(drop=True)], axis=1)
X_poly_combined.columns = X_poly_combined.columns.astype(str)

# Combine numerical and categorical features for other models
X_combined = pd.concat([pd.DataFrame(X, columns=numerical_features), data[categorical_features].reset_index(drop=True)], axis=1)
X_combined.columns = X_combined.columns.astype(str)

# Define parameter grids for each model
param_grid_logistic = {
    'C': [0.01, 0.1, 1, 10, 100],
    'class_weight': [{0: 1, 1: 2}, {0: 1, 1: 3}, 'balanced']
}

param_grid_random_forest = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

param_grid_gradient_boosting = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5]
}

param_grid_decision_tree = {
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialise models
models = {
    'Logistic Regression': LogisticRegression(max_iter=3000),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'Decision Tree': DecisionTreeClassifier()
}

param_grids = {
    'Logistic Regression': param_grid_logistic,
    'Random Forest': param_grid_random_forest,
    'Gradient Boosting': param_grid_gradient_boosting,
    'Decision Tree': param_grid_decision_tree
}

all_results = {
    'discovery_set': {},
    'validation_set1': {},
    'validation_set2': {},
    'validation_set3': {}
}

for model_name in models:
    for dataset in all_results:
        all_results[dataset][model_name] = {
            'accuracy': [],
            'roc_auc': [],
            'aupr': [],
            'sensitivity': [],
            'f1_0': [],
            'f1_1': []
        }

# Complete hyperparameter search, train models 10 times, and evaluate
for run in range(1, 11):
    print(f"Run {run}")

    # Apply SMOTENC to balance the dataset with different random state each run
    smotenc = SMOTENC(categorical_features=[X_poly_combined.columns.get_loc(col) for col in categorical_features], random_state=run)
    X_resampled_poly, y_resampled_poly = smotenc.fit_resample(X_poly_combined, y)

    smotenc = SMOTENC(categorical_features=[X_combined.columns.get_loc(col) for col in categorical_features], random_state=run)
    X_resampled_combined, y_resampled_combined = smotenc.fit_resample(X_combined, y)

    # Initialise models with different random state each run
    models = {
        'Logistic Regression': LogisticRegression(max_iter=3000, random_state=run),
        'Random Forest': RandomForestClassifier(random_state=run),
        'Gradient Boosting': GradientBoostingClassifier(random_state=run),
        'Decision Tree': DecisionTreeClassifier(random_state=run)
    }

    best_models = {}

    # Perform hyperparameter search and fit models
    for model_name in models:
        print(f"Training and hyperparameter tuning for {model_name}...")
        if model_name == 'Logistic Regression':
            best_model = hyperparameter_search(models[model_name], param_grids[model_name], X_resampled_poly, y_resampled_poly, random_state=run)
        else:
            best_model = hyperparameter_search(models[model_name], param_grids[model_name], X_resampled_combined, y_resampled_combined, random_state=run)

        best_models[model_name] = best_model

        # Evaluate models with specified thresholds
        if model_name == 'Logistic Regression':
            threshold = 0.55
            X_eval = X_poly_combined
        else:
            threshold = 0.5
            X_eval = X_combined

        print(f"Evaluation on the discovery set for {model_name} with threshold {threshold}:")
        discovery_metrics = evaluate_model(best_model, X_eval, y, threshold=threshold)
        for metric in all_results['discovery_set'][model_name]:
            all_results['discovery_set'][model_name][metric].append(discovery_metrics[metric])

        # Print metrics for each run
        print(f"Run {run} - Discovery Set - {model_name} Metrics:")
        for metric_name, metric_value in discovery_metrics.items():
            print(f"{metric_name}: {metric_value}")

    # Paths to the validation sets
    validation_sets = {
        'validation_set1': '/content/drive/MyDrive/Liver Fibrosis/validation_set1.csv',
        'validation_set2': '/content/drive/MyDrive/Liver Fibrosis/validation_set2.csv',
        'validation_set3': '/content/drive/MyDrive/Liver Fibrosis/validation_set3.csv'
    }

    # Evaluate each best model on each validation set
    for validation_set_name, validation_set_path in validation_sets.items():
        validation_data = pd.read_csv(validation_set_path)

        # Apply the transformation to the validation set
        validation_data = transform_categorical_columns(validation_data)

        # Drop all other columns except the selected features and the target
        validation_data = validation_data[features_to_keep]

        # Separate features 'X_val' and target 'y_val'
        X_val = validation_data.drop(columns=['group'])
        y_val = validation_data['group']

        # Standardise numerical features using the same scaler fitted on the discovery set
        X_val[numerical_features] = scaler.transform(X_val[numerical_features])

        # Generate polynomial features for Logistic Regression
        X_val_poly = poly.transform(X_val[numerical_features])
        X_val_poly_df = pd.DataFrame(X_val_poly, columns=poly.get_feature_names_out(numerical_features))
        X_val_poly_combined = pd.concat([X_val_poly_df, validation_data[categorical_features].reset_index(drop=True)], axis=1)
        X_val_poly_combined.columns = X_val_poly_combined.columns.astype(str)

        # Combine numerical and categorical features for other models
        X_val_combined = pd.concat([pd.DataFrame(X_val, columns=numerical_features), validation_data[categorical_features].reset_index(drop=True)], axis=1)
        X_val_combined.columns = X_val_combined.columns.astype(str)

        for model_name in best_models:
            if model_name == 'Logistic Regression':
                threshold = 0.55
                X_eval = X_val_poly_combined
            else:
                threshold = 0.5
                X_eval = X_val_combined

            print(f"Evaluation on {validation_set_name} for {model_name} with threshold {threshold}:")
            validation_metrics = evaluate_model(best_models[model_name], X_eval, y_val, threshold=threshold)
            for metric in all_results[validation_set_name][model_name]:
                all_results[validation_set_name][model_name][metric].append(validation_metrics[metric])

            # Print metrics for each run
            print(f"Run {run} - {validation_set_name} - {model_name} Metrics:")
            for metric_name, metric_value in validation_metrics.items():
                print(f"{metric_name}: {metric_value}")

# Calculate mean and standard dev for each metric
final_results = {}

for dataset_name, dataset_results in all_results.items():
    final_results[dataset_name] = {}
    for model_name, metrics in dataset_results.items():
        final_results[dataset_name][model_name] = {}
        for metric_name, values in metrics.items():
            final_results[dataset_name][model_name][f'{metric_name}_mean'] = np.mean(values)
            final_results[dataset_name][model_name][f'{metric_name}_std'] = np.std(values)

# Print final results
for dataset_name, dataset_results in final_results.items():
    print(f"Final Results for {dataset_name}:")
    for model_name, metrics in dataset_results.items():
        print(f"Model: {model_name}")
        for metric_name, value in metrics.items():
            print(f"{metric_name}: {value}")
        print("\n")


Run 1
Training and hyperparameter tuning for Logistic Regression...
Fitting 5 folds for each of 15 candidates, totalling 75 fits
Evaluation on the discovery set for Logistic Regression with threshold 0.55:
Run 1 - Discovery Set - Logistic Regression Metrics:
accuracy: 0.7925925925925926
confusion_matrix: [[320  71]
 [ 41 108]]
roc_auc: 0.848744400006866
aupr: 0.7076429264776841
sensitivity: 0.7248322147651006
f1_0: 0.8510638297872342
f1_1: 0.6585365853658537
Training and hyperparameter tuning for Random Forest...
Fitting 5 folds for each of 81 candidates, totalling 405 fits
Evaluation on the discovery set for Random Forest with threshold 0.5:
Run 1 - Discovery Set - Random Forest Metrics:
accuracy: 1.0
confusion_matrix: [[391   0]
 [  0 149]]
roc_auc: 1.0
aupr: 1.0
sensitivity: 1.0
f1_0: 1.0
f1_1: 1.0
Training and hyperparameter tuning for Gradient Boosting...
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Evaluation on the discovery set for Gradient Boosting with thresh